In [1]:
%reload_ext autoreload
%autoreload 2


# Quick Start

To run a simulation on a terrain stored as an heightmap and store the robot's information in a bag file. Remember to throttle the `/pose` topic with
`rosrun topic_tools throttle messages /krock/pose 50`

In [3]:
from simulation import Simulation
out_dir='/home/francesco/Desktop/test/'
# run two simulations on the bumps0 map for 10 seconds each
sim = Simulation(map_path='./maps/new-train/bumps0.png', max_time=10, n=2, out_dir=out_dir, height=1)
sim()
print('[INFO] bags stored at {}'.format(sim.agent_out_dir))

/home/francesco/Documents/Master-Thesis/core/simulation/env/webots/krock/children_no_tail


  0%|          | 0/2 [00:00<?, ?it/s]

success: 1


Reanimate robot:  50%|█████     | 1/2 [00:12<00:12, 12.19s/it]

success: 1


[INFO] bags stored at /home/francesco/Desktop/test//bags


You can also pass a `spawn_strategy` that decides how the robot will spawn on the ground. We provided a [`RandomSpawnStrategy`](https://github.com/FrancescoSaverioZuppichini/Master-Thesis/blob/master/core/simulation/env/spawn/SpawnStragety.py) and a [`FlatGroundSpawnStrategy`](https://github.com/FrancescoSaverioZuppichini/Master-Thesis/blob/master/core/simulation/env/spawn/SpawnStragety.py). The second will spawn the robot on the flat part of the terrain.

The previous snippet will run two simulations for 10 second each on the provided map. Krock will walk on the terrain until the time finish or it reach the boundaries of the terrain. The simulation information are stored in a `.csv` file at `out_dir/meta.csv`.

In [4]:
import pandas as pd

pd.read_csv( out_dir + 'meta.csv')

,Unnamed: 0,filename,map,height
0,0,bumps0-1-1563102050.8296146,bumps0,1
1,1,bumps0-1-1563102061.9010878,bumps0,1


The `.bag` files are stored into `tmp`

In [6]:
!ls /home/francesco/Desktop/test/bags | grep .bag

bumps0-1-1563102050.8296146.bag
bumps0-1-1563102061.9010878.bag


## Customization
To change how the robot behaviour you can create subclass the `Simulation` class and override `step`

In [3]:
from random import randint

class SimulationWithRandomGait(Simulation):
    def step(self):
        return self.env.step({
        'frontal_freq': 1,
        'lateral_freq': 0,
        'gait': randint(1,3) # this will select a different gait at each step
    })
    
sim = SimulationWithRandomGait(map_path='./maps/new-train/bumps0.png', max_time=10, n=2, out_dir='/tmp/', height=1)
sim()

  0%|          | 0/2 [00:00<?, ?it/s]

success: 1


Reanimate robot:  50%|█████     | 1/2 [00:12<00:12, 12.20s/it]

AttributeError: 'NoneType' object has no attribute 'field'

To understand why and how this work please read the following sections.

# API

This packages provide a clean API to create and run a robotic simulation. In our work, we used Webots as engine and a robot called Krock. 

## Webots + Krock
![alt text](https://raw.githubusercontent.com/FrancescoSaverioZuppichini/Master-Thesis/master/resources/images/krock.jpg)

In order to run the Krock robot using Webots we provide a [gym](https://gym.openai.com/) env called `KrockWebotsEnv`. If you know gym, then you are good to go. It follows a **basic example** where we let Krock walk forward

In [35]:
import rospy
import pprint

from simulation.env.webots.krock import Krock, KrockWebotsEnv
# load a world file
WORLD_PATH = '/home/francesco/Documents/Master-Thesis/core/env/webots/krock/krock2_ros/worlds/holes2.wbt'
N_STEPS = 4

rospy.init_node("traversability_simulation")
# create our env
env = KrockWebotsEnv.from_file(WORLD_PATH)
# spawn the robot at random pose
init_obs = env.reset(pose=env.random_position)

print('Initial observations:')
pprint.pprint(init_obs)

for _ in range(N_STEPS):
#   go forward for a while
    obs, r, done, _ = env.step(env.GO_FORWARD)
    pprint.pprint(obs)
    if done: break
        
env.step(env.STOP)

success: 1
Initial observations:
{'sensors': {'front_cam': None,
             'orientation': {'w': 0.1856268810151021,
                             'x': 0.018228798012318075,
                             'y': 0.06653953354160076,
                             'z': -0.9801953185175379},
             'position': {'x': 0.2315982956667584,
                          'y': -0.1937481574050184,
                          'z': -0.5933946612326868}}}
{'sensors': {'front_cam': None,
             'orientation': {'w': 0.1856268810151021,
                             'x': 0.018228798012318075,
                             'y': 0.06653953354160076,
                             'z': -0.9801953185175379},
             'position': {'x': 0.2315982956667584,
                          'y': -0.1937481574050184,
                          'z': -0.5933946612326868}}}
{'sensors': {'front_cam': None,
             'orientation': {'w': 0.1856268810151021,
                             'x': 0.018228798012318075,
     

({'sensors': {'position': {'x': 0.2315982956667584,
    'y': -0.1937481574050184,
    'z': -0.5933946612326868},
   'orientation': {'x': 0.018228798012318075,
    'y': 0.06653953354160076,
    'z': -0.9801953185175379,
    'w': 0.1856268810151021},
   'front_cam': None}},
 0,
 False,
 {})

### Actions

`KrockWebotsEnv` implements `WebotsEnv` that is a generic env for Webots. Thanks to to gym interface we can explore the action space and the observation space easily



In [26]:
print(env.action_space)
print(env.observation_space)

Dict(frontal_freq:Box(), gait:Discrete(2), lateral_freq:Box())
Dict(sensors:Dict(front_cam:Tuple(Box(600, 800, 3), Box(600, 800, 3)), orientation:Dict(w:Box(1,), x:Box(1,), y:Box(1,), z:Box(1,)), position:Dict(x:Box(1,), y:Box(1,), z:Box(1,))))


Select a random action is even esier


In [43]:
obs, r, done, _ = env.step(env.action_space.sample())
env.step(env.STOP)

({'sensors': {'position': {'x': -1.403106178441482,
    'y': -4.495554941675828,
    'z': -0.6178344141800893},
   'orientation': {'x': -0.0903590497756354,
    'y': 0.024031471713933404,
    'z': 0.8236676444897191,
    'w': 0.5593114891647272},
   'front_cam': None}},
 0,
 False,
 {})

We can easily **change the gait** by passing the correct action
![alt text](https://raw.githubusercontent.com/FrancescoSaverioZuppichini/Master-Thesis/master/resources/images/krock-gait.jpg)


In [57]:
import time
# init_obs = env.reset(pose=env.random_position)
obs, r, done, _ = env.step({
        'frontal_freq': 1,
        'lateral_freq': 0,
        'gait': 2 # change gait to 2
    })
time.sleep(2)
obs, r, done, _ = env.step(env.STOP) # stop will also reset the gait to 1

shutdown request: new node registered with same name


# Under the hood

To create the final enviroment we had to first.

## Define the Enviroment
Under the hood `KrockWebotsEnv` implements [`WebotsEnv`](https://github.com/FrancescoSaverioZuppichini/Master-Thesis/blob/master/core/simulation/env/webots/WebotsEnv.py). `WebotsEnv` creates the enviroment by from an image (heightmap)/ numpy array or a webots worl file. Check the `classmethod`. It also allows to get the height in a point and to spawn the robot in the terrain. 

## Define the Robot

### Agent API
The base class to create a robot is `Agent`

In [66]:
from simulation.agent import Agent

class DummyAgent(Agent):
    def __call__(self, *args, **kwargs):
        print('I am alive!')
    def act(self, env, *args, **kwargs):
        print('I am moving!')
    def die(self, *args, **kwargs):
        print('bye bye')
    
a = DummyAgent()
env = None
a() # init
a.act(env) # do something
a.die(env) # RIP, clean up (e.g store stuff empty the state)

I am alive!
I am moving!
bye bye


### ROS
A `Rospy` agent is already implemented and can be found at `agent.RospyAgent`. It adds two methods to the basic `Agent`: `init_publishers` and `init_subscribers`. They must be overrided to initialise the subscribers and listeners. 

In [68]:
from simulation.agent import RospyAgent
from geometry_msgs.msg import PoseStamped
from sensor_msgs.msg import Joy

class MyRospyAgent(RospyAgent):
    def init_publishers(self):
        return {
            'joy': rospy.Publisher('/joy', Joy, queue_size=1),
        }

    def init_subscribers(self):
        return { 'pose': 
                rospy.Subscriber('/pose', PoseStamped, self.callback_pose)
               }

    def callback_pose(self, data):
        # this will triggers the `on_state_change` method in all callbacks
        self.state['pose'] = data

a = MyRospyAgent()
a()
print(a.subscribers)
print(a.publishers)


{'pose': <rospy.topics.Subscriber object at 0x7fe8a9e37438>}
{'joy': <rospy.topics.Publisher object at 0x7fe80c5168d0>}


### Callbacks
`Agent` implements the protocol `Callbackable` protocol. A callback can be
added by calling the `add_callback` or `add_callbacks`. 

In [69]:
from simulation.agent.callbacks import AgentCallback

class MyCallback(AgentCallback):
    def on_state_change(self, agent, key, value):
        print(key, value)

Each `Agent` has a `state` that is just a `dict` . Every time it is changed it triggers the `on_state_change` event. This is usefull
to record stuff.

In [70]:
a = DummyAgent()
a.add_callback(MyCallback())
a()

a.state['foo'] = 'foo'

I am alive!
foo foo


#### RosBagSaver

A callback to store the topics of an agent into bags file is implemented

In [71]:
from simulation.agent.callbacks import RosBagSaver

a = MyRospyAgent()
a.add_callbacks([RosBagSaver(save_dir='/temp/data', topics=['pose'])])
a()

The files will be stored everytime `agent.die()` il called. A new thread is always spawned to avoid blocking

## Krock
[`Krock`](https://github.com/FrancescoSaverioZuppichini/Master-Thesis/blob/master/core/simulation/env/webots/krock/Krock.py) is just a `RospyAgent` that has already all the subscribers and listeners.